In [5]:
!pip install mediapipe
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import math
from sklearn.model_selection import train_test_split

In [4]:
labels = ['bhujangasana', 'padmasana', 'savasana', 'tadasana', 'trikonasana', 'vrikshasana']

# Function to load and preprocess data
def load_data():
    base_path = "C:\\Users\\rathi\\Downloads\\Design Project\\yoga-lstm\\kaggle\\input\\yoga-lstm\\kaggle\\working\\MP_Data"
    
    all_label_data = []

    for label in labels:
        asana_path = os.path.join(base_path, label)
        label_data = []

        for file in os.listdir(asana_path):
            file_path = os.path.join(asana_path, file)
            df = pd.read_csv(file_path)  # Assuming CSV contains the sequence data
            label_data.append(df)

        all_label_data.append(label_data)

    return all_label_data

In [5]:
# Function to preprocess sequences
def preprocess_sequences(sequences):
    sequence_length = 125
    num_features = 6
    processed_sequences = []

    for label_data in sequences:
        padded_sequences = []
        for seq in label_data:
            seq_data = seq.to_numpy()
            if len(seq_data) >= sequence_length:
                padded_seq = seq_data[:sequence_length, :]
            else:
                padded_seq = np.zeros((sequence_length, num_features))
                padded_seq[:len(seq_data), :] = seq_data

            padded_sequences.append(padded_seq)
        processed_sequences.append(padded_sequences)

    return processed_sequences

In [6]:
# Function to prepare data for training
def prepare_data(all_label_data):
    X_data = []
    y_data = []
    
    for i, label_data in enumerate(all_label_data):
        for seq in label_data:
            X_data.append(seq[:, :-1])
            y_data.append(i)

    return np.array(X_data), np.array(y_data)

In [7]:
# Load and preprocess data
all_label_data = load_data()
processed_sequences = preprocess_sequences(all_label_data)

# Prepare data for training
X, y = prepare_data(processed_sequences)

In [8]:
X[0]

array([[193.3942882226961, 220.8160550900032, 212.4204192408847,
        163.37622371284388, 262.60227556667155, 247.04169509847844],
       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
       [165.14639178624162, 174.71373227714116, 25.09001439013326,
        327.6783221914968, 196.4396568352307, 191.1949309465426],
       [164.14170948359427, 174.1107089648937, 27.385092451136845,
        326.79136168406194, 197.72972156656337, 192.28347987180064],
       [164.61422985002022, 173.65222485932082, 27.900124815627056,
        326.1512343406088, 198.5079291161747, 193.13003993390868],
       [166.0933076203305, 170.10835070436502, 27.434859479822105,
        325.1031867890636, 198.57067944539605, 193.94771937932865],
       [167.18671827724737, 168.39100815434222, 27.02511255761749,
        324.83546528838065, 198.5129402932313, 190.3731136240121],
       [167.0308416483443, 167.39967190570826, 27.025297616176456,
        324.74288231643305, 198.78447954163505, 187.2355392227843],
       [166.6647358

In [9]:
from keras.utils import to_categorical

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Convert the input data to the appropriate data type
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
# Ensure y_train and y_test are one-hot encoded for multi-class classification
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [10]:
len(X_train[0])

125

In [11]:
X_train[0]

array([[307.1561  ,  57.36809 ,  21.630146,  26.381855, 287.38885 ,
        176.48862 ],
       [317.16934 ,  44.092846,  27.908066,  20.283075, 183.58832 ,
        181.64998 ],
       [315.38364 ,  47.778145,  24.974625,  17.487305, 185.49213 ,
        184.31966 ],
       [316.03952 ,  45.603436,  24.875896,  18.534172, 187.30188 ,
        186.79703 ],
       [316.30682 ,  45.043262,  24.2854  ,  19.769135, 184.82858 ,
        188.65758 ],
       [317.07147 ,  43.268993,  25.736734,  20.726873, 181.15884 ,
        187.62341 ],
       [317.09598 ,  42.55347 ,  26.325434,  20.568867, 179.50452 ,
        186.69421 ],
       [316.03455 ,  41.72108 ,  27.319485,  21.095406, 177.86801 ,
        182.89407 ],
       [315.71973 ,  41.88218 ,  27.381248,  20.797554, 178.938   ,
        178.4937  ],
       [315.31012 ,  41.919262,  27.111095,  20.428545, 178.55814 ,
        178.1054  ],
       [314.78372 ,  41.37369 ,  27.066471,  20.225018, 178.69196 ,
        173.42783 ],
       [314.5546  ,  

In [12]:
y_train_encoded[0]

array([0., 0., 0., 0., 0., 1.], dtype=float32)

In [13]:
labels[y_train[0]]

'vrikshasana'

In [21]:
y_train[0]

5

In [22]:
len(X_train)

115

In [23]:
len(y_train_encoded)

115

In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [15]:
# Create a Sequential model
model = Sequential()

# Add LSTM layers with dropout and batch normalization
model.add(LSTM(units=128, return_sequences=True, input_shape=(125, 6)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Add a Dense output layer for classification
model.add(Dense(units=6, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 125, 128)          69120     
                                                                 
 dropout (Dropout)           (None, 125, 128)          0         
                                                                 
 batch_normalization (Batch  (None, 125, 128)          512       
 Normalization)                                                  
                                                                 
 lstm_1 (LSTM)               (None, 125, 128)          131584    
                                                                 
 dropout_1 (Dropout)         (None, 125, 128)          0         
                                                                 
 batch_normalization_1 (Bat  (None, 125, 128)          512       
 chNormalization)                                       

In [16]:
model.fit(X_train, y_train_encoded, epochs=100, callbacks=[tb_callback])

Epoch 1/100
4/4 [==============================] - 14s 381ms/step - loss: 1.3759 - accuracy: 0.5217
Epoch 2/100
4/4 [==============================] - 2s 385ms/step - loss: 0.1884 - accuracy: 0.9478
Epoch 3/100
4/4 [==============================] - 2s 389ms/step - loss: 0.1422 - accuracy: 0.9565
Epoch 4/100
4/4 [==============================] - 2s 394ms/step - loss: 0.0599 - accuracy: 0.9826
Epoch 5/100
4/4 [==============================] - 2s 383ms/step - loss: 0.0284 - accuracy: 1.0000
Epoch 6/100
4/4 [==============================] - 2s 392ms/step - loss: 0.0319 - accuracy: 0.9913
Epoch 7/100
4/4 [==============================] - 2s 390ms/step - loss: 0.0467 - accuracy: 0.9826
Epoch 8/100
4/4 [==============================] - 2s 388ms/step - loss: 0.0167 - accuracy: 1.0000
Epoch 9/100
4/4 [==============================] - 2s 393ms/step - loss: 0.0516 - accuracy: 0.9826
Epoch 10/100
4/4 [==============================] - 2s 382ms/step - loss: 0.0153 - accuracy: 1.0000
Epoch 11

Epoch 83/100
4/4 [==============================] - 2s 400ms/step - loss: 0.1370 - accuracy: 0.9652
Epoch 84/100
4/4 [==============================] - 2s 400ms/step - loss: 0.0428 - accuracy: 0.9826
Epoch 85/100
4/4 [==============================] - 2s 408ms/step - loss: 0.0313 - accuracy: 0.9913
Epoch 86/100
4/4 [==============================] - 2s 412ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 87/100
4/4 [==============================] - 2s 415ms/step - loss: 0.0268 - accuracy: 0.9913
Epoch 88/100
4/4 [==============================] - 2s 388ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 89/100
4/4 [==============================] - 2s 402ms/step - loss: 0.0061 - accuracy: 1.0000
Epoch 90/100
4/4 [==============================] - 2s 410ms/step - loss: 0.0137 - accuracy: 0.9913
Epoch 91/100
4/4 [==============================] - 2s 408ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 92/100
4/4 [==============================] - 2s 407ms/step - loss: 0.0640 - accuracy: 0.9739


In [17]:
y_pred=model.predict(X_test)
predicted_labels = np.argmax(y_pred, axis=1)
predicted_labels

1/1 [==============================] - 2s 2s/step


array([4, 0, 3, 4, 2, 0, 5, 2, 2, 0, 2, 3, 3, 5, 4, 2, 1, 2, 5, 3, 0, 4,
       4, 3, 2, 1, 0, 0, 1], dtype=int64)

In [18]:
y_test

array([4, 0, 3, 4, 2, 0, 5, 2, 2, 0, 2, 3, 3, 5, 4, 2, 1, 2, 5, 3, 0, 4,
       4, 4, 2, 1, 0, 0, 1])

In [19]:
x=0
for i in range(len(y_test)):
    if y_test[i]==predicted_labels[i]:
        x+=1
print(x/len(y_test))

0.9655172413793104


In [20]:
model.save('lstm25fps.h5')

C:\Users\rathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('lstm25fps.h5')

# Webcam Integration

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [8]:
def calculateAngle(landmark1, landmark2, landmark3):
    x1, y1, z1 = landmark1
    x2, y2, z2 = landmark2
    x3, y3, z3 = landmark3
    
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    if angle < 0:
        angle += 360
    
    return angle

In [9]:

def classifyAngles_check(landmarks):
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])

#     print(f"Left Elbow Angle: {left_elbow_angle}, "
#           f"Right Elbow Angle: {right_elbow_angle}, "
#           f"Left Shoulder Angle: {left_shoulder_angle}, "
#           f"Right Shoulder Angle: {right_shoulder_angle}, "
#           f"Left Knee Angle: {left_knee_angle}, "
#           f"Right Knee Angle: {right_knee_angle}")
    
    return [left_elbow_angle,right_elbow_angle,left_shoulder_angle,right_shoulder_angle,left_knee_angle,right_knee_angle]


In [10]:
import cv2
import mediapipe as mp
import numpy as np

labels = ['bhujangasana', 'padmasana', 'savasana', 'tadasana', 'trikonasana', 'vrikshasana']

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, model_complexity=1)

# Open the webcam
cap = cv2.VideoCapture(0)

frame_count = 0
frames_data = []
capture_frames = False
prediction = ""

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # If 'c' is pressed, start capturing frames
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        capture_frames = not capture_frames  # Toggle capturing frames

        # Reset frame count and frames_data when starting to capture frames
        if capture_frames:
            frame_count = 0
            frames_data = []
    
    if capture_frames:
        # Process the frame to detect poses
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        if results.pose_landmarks:
            # Draw the detected poses on the frame
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            height, width, _ = frame.shape
            rows = 33
            cols = 3
            landmarks = [[0 for _ in range(cols)] for _ in range(rows)]
            landmark = results.pose_landmarks.landmark

            # Get pose landmarks and store them for prediction
            for i in range(len(landmark)):
                landmarks[i] = ((landmark[i].x * width, landmark[i].y * height, landmark[i].z * width))
            angles = classifyAngles_check(landmarks)

            frames_data.append(angles)
            frame_count += 1

            # If 125 frames captured, perform prediction
            if frame_count == 125:
                # Convert frames_data to numpy array for prediction (shape: (125, 33, 3))
                frames_array = np.array(frames_data)
                frames_array = frames_array.reshape((1, 125, 6))

                # Perform prediction
                output = loaded_model.predict(frames_array)
                predicted_label = np.argmax(output)
                prediction = labels[predicted_label]

                frame_count = 0
                frames_data = []

    # Display prediction on the OpenCV window
    cv2.putText(frame, f"Prediction: {prediction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Pose Detection', frame)

    if key == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 49ms/step


In [ ]:
cap = cv2.VideoCapture(0)
